# Weak lensing – Notebook 29th May 2024

Notebook tested with commit nb `XXX`.

Check requirements in `env.yml` (conda virtual environment).

The `pycs` package from the `cosmostat` repository (https://github.com/CosmoStat/cosmostat) could not be installed with `pip install .`. Therefore, the path to the git repository must be added to `PYTHONPATH` (see below). Tested with commit nb `a420193ced4bcd081f04a0afe0773aa4c120bfc8` (branch "hos").

The `pysparse` package from the `Sparse2D` repository  (https://github.com/CosmoStat/Sparse2D) needs to be manually installed. Tested with commit nb `3f9d54863765980299cfe92e0624ba93ed7ff02b`.

Very dirty workaround to avoid `ImportError: libpython3.11.so.1.0: cannot open shared object file: No such file or directory`: create a symbolic link from the `libpython3.11.so.1.0` file, which is typically stored in the `~/miniconda3/envs/wl-pycs/lib` (i.e., the directory for shared libraries within the virtual environment), to a standard root location such as `/lib/x86_64-linux-gnu`.

In [1]:
%config InlineBackend.figure_format = 'svg'
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import pickle

import numpy as np
from scipy import stats, optimize
import matplotlib.pyplot as plt

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

import weaklensing as wl
import weaklensing.kappatng as wlktng
import weaklensing.cosmos as wlcosmos
import weaklensing.utils as wlutils
import weaklensing.cqr as wlcqr
import weaklensing.rcps as wlrcps

pycs_dir = os.path.expanduser(wl.CONFIG_DATA['pycs_dir'])
sys.path.append(pycs_dir) # tested with commit nb b7b39124de0ad444a89e27123e1791400edfe748 (branch "hos-hl")
print(f"The `pycs` package is located in {pycs_dir}")

import pycs.astro.wl.mass_mapping as csmm

Configuration file found in /home/leterme231/OneDrive/Documents/Code/weaklensing
The `pycs` package is located in /home/leterme231/OneDrive/Documents/Code/cosmostat


### Compare kappaTNG datasets

Old sample dataset vs full dataset in `hdf5` format.

In [3]:
size = 1.5

In [4]:
width, size = wlktng.get_npixels_openingangle(size)
print(f"Nb of pixels per side = {width}")
print(f"Opening angle = {size:.2f} deg")

Nb of pixels per side = 306
Opening angle = 1.49 deg


Load data from the COSMOS catalog

In [5]:
cat_cosmos = wlcosmos.cosmos_catalog(
    include_faint=False
)
data_cosmos = wlcosmos.get_data_from_cosmos(cat_cosmos, size)
extent = data_cosmos["extent"]

Load data from the full dataset

In [10]:
# Instantiate KappaTNG object
ktng = wlktng.KappaTNG(size=size)